# ***MAIN FUNCTIONALITY***

In [1]:
import plotly.express as px
import seaborn as sns

In [1]:
from meteo.Site import Site
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging
from misc.util import load_pv_model

configure_logging()

# Set up site location and get TMY data
name = ""
address = "York St, Belfast, BT15 1ED"
client = "Ulster University"

site = Site(name=name, 
            address=address, 
            client=client, 
            latitude=54.60452, 
            longitude=-5.92860, 
            size=100)

# Set up PV arrays
pv_kwp = 1
surface_pitch = 35
surface_azimuth = -90
lifespan = 25
pv_eol_derating = 0.88
pv_cost = 1250
efficiency = 0.21
temp_coeff = -0.0035
transmission_absorption = 0.9
refraction_index = 0.1

array_1 = SolarPVArray(pv_kwp, surface_pitch, surface_azimuth, lifespan, pv_eol_derating,
                       pv_cost, efficiency, temp_coeff, transmission_absorption, refraction_index)

array_2 = SolarPVArray(1, 35, -45, cost_per_kWp=pv_cost)
array_3 = SolarPVArray(1, 35, 0, 25, 0.88)
array_4 = SolarPVArray(1, 35, 45)
array_5 = SolarPVArray(1, 35, 90)

# Arrange all arrays into list for modelling
arrays = [array_1, array_2, array_3, array_4, array_5]

# Run SolarPV model
pv_model = SolarPVModel(site, arrays)

# Save model as .wmm file
pv_model.save_model()

# Load .wwm model from file
pv_model = load_pv_model()


2024-02-17 22:11:22,023 - INFO - Fetching TMY data for latitude: 54.60452, longitude: -5.9286
2024-02-17 22:11:23,024 - INFO - HTTP Request: GET https://re.jrc.ec.europa.eu/api/tmy?lat=54.60452&lon=-5.9286&startyear=2005&endyear=2015&outputformat=json "HTTP/1.1 200 OK"
2024-02-17 22:11:24,059 - INFO - TMY data obtained for: 54.60452, longitude: -5.9286
2024-02-17 22:11:24,060 - INFO - *******************
2024-02-17 22:11:24,061 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: -90deg, Lifespan: 25yrs
2024-02-17 22:11:24,061 - INFO - *******************
2024-02-17 22:11:24,062 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: -45deg, Lifespan: 25yrs
2024-02-17 22:11:24,062 - INFO - *******************
2024-02-17 22:11:24,062 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 0deg, Lifespan: 25yrs
2024-02-17 22:11:24,063 - INFO - *******************
2024-02-17 22:11:24,064 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth:

In [26]:
pv_model.summary_grouped.quarterly

In [ ]:
px.bar(pv_model.summary_grouped.daily.PV_Gen_kWh_Total)

# ***Performance Test***

In [2]:
# Performance Testing
import pstats
import cProfile
cProfile.run("SolarPVModel(site, [array_1])", 'performance_test.prof')
p = pstats.Stats('performance_test.prof')
p.sort_stats('tottime').print_stats()
p.sort_stats('cumtime').print_stats()

2024-02-17 22:11:26,005 - INFO - *******************
2024-02-17 22:11:26,006 - INFO - Starting Solar PV model simulations.
2024-02-17 22:11:26,007 - INFO - *******************
2024-02-17 22:11:26,007 - INFO - Simulating model - PV Size: 1kWp, Pitch: 35 degrees, Azimuth -90 degrees WoS
2024-02-17 22:11:26,016 - INFO - *******************
2024-02-17 22:11:26,017 - INFO - Solar PV model simulations completed.
2024-02-17 22:11:26,043 - INFO - Solar PV model data aggregated.
2024-02-17 22:11:26,052 - INFO - Solar PV model data summary complete.
2024-02-17 22:11:26,053 - INFO - *******************
2024-02-17 22:11:26,055 - INFO - Solar PV model statistical analysis completed.
2024-02-17 22:11:26,082 - INFO - Solar PV model statistical grouping completed.
2024-02-17 22:11:26,083 - INFO - *******************


Sat Feb 17 22:11:26 2024    performance_test.prof

         62511 function calls (61533 primitive calls) in 0.080 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.005    0.000    0.012    0.001 C:\Users\djp12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\frame.py:11093(_reduce)
       55    0.003    0.000    0.004    0.000 C:\Users\djp12\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\groupby\ops.py:348(_call_cython_op)
       11    0.003    0.000    0.005    0.000 {pandas._libs.lib.map_infer}
       96    0.003    0.000    0.003    0.000 {method 'acquire' of '_thread.lock' objects}
        8    0.002    0.000    0.002    0.000 c:\Users\djp12\Documents\Coding\Github_Repos\WiseWattage\WiseWattage\solar\solar_radiation.py:16(calc_time_co

# ***Example Running Model with variable inputs***

In [2]:
# %%timeit

from meteo.Site import Site
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging

configure_logging()

# Set up site and obtain TMY data
site = Site(name="Test Site", address = "123 Test Street", client = "Big Gerry", 
            latitude=54.60452, longitude=-5.92860, tmz_hrs_east = 0)

2024-02-17 15:50:10,551 - INFO - TMY data obtained for: 54.60452, longitude: -5.9286
2024-02-17 15:50:10,552 - INFO - *******************


In [7]:
# %%timeit

pv_kwp = 1
surface_pitch = 35
surface_azimuth = -90
lifespan = 25
pv_eol_derating = 0.88

# Set up PV arrays
array_1 = SolarPVArray(pv_kwp, surface_pitch, surface_azimuth, lifespan, pv_eol_derating)
array_2 = SolarPVArray(1, 35, -45, 25, 0.88)
array_3 = SolarPVArray(1, 35, 0, 25, 0.88)
array_4 = SolarPVArray(1, 35, 45, 25, 0.88)
array_5 = SolarPVArray(1, 35, 90, 25, 0.88)
arrays = [array_1, array_2, array_3, array_4, array_5]

# Set up and run model
pv_model = SolarPVModel(site=site, arrays=arrays)

2024-02-17 15:51:20,242 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: -90deg, Lifespan: 25yrs
2024-02-17 15:51:20,243 - INFO - *******************
2024-02-17 15:51:20,243 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: -45deg, Lifespan: 25yrs
2024-02-17 15:51:20,243 - INFO - *******************
2024-02-17 15:51:20,244 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 0deg, Lifespan: 25yrs
2024-02-17 15:51:20,244 - INFO - *******************
2024-02-17 15:51:20,245 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 45deg, Lifespan: 25yrs
2024-02-17 15:51:20,245 - INFO - *******************
2024-02-17 15:51:20,245 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 90deg, Lifespan: 25yrs
2024-02-17 15:51:20,246 - INFO - *******************
2024-02-17 15:51:20,246 - INFO - *******************
2024-02-17 15:51:20,246 - INFO - Starting Solar PV model simulations.
2024-02-17 15:51:20,247 - INFO - ********

In [ ]:
101 ms ± 3.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
100 ms ± 3.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
101 ms ± 3.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [ ]:
195 ms ± 4.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
198 ms ± 1.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
193 ms ± 4.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
200 ms ± 6.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# ***Visualisation***

In [ ]:
import plotly.express as px
import seaborn as sns

In [ ]:
pv_model.array_model(1).head()

In [ ]:
print(f"PV Gen: {round(pv_model.all_models.PV_Gen_kWh_Array_1.sum(), 3)}kWh")
print(f"E POA: {round(pv_model.all_models.E_POA_kWm2_Array_1.sum(), 3)}kWhm2")

In [ ]:
px.bar(pv_model.summary_grouped.hourly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.weekly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.daily[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["Cell_Temp_C", "T2m"]])


In [ ]:
sns.lineplot(pv_model.models[0]["model_result"][["Cell_Temp_C", "T2m"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

In [ ]:
sns.lineplot(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

# ***Mass Model Testing***

In [ ]:
# %%timeit
import random

# Assuming the Site, SolarPVArray, and SolarPVModel classes are defined elsewhere
site = Site(name="Test Site", address="123 Test Street", client="Big Gerry", 
            latitude=54.60452, longitude=-5.92860, tmz_hrs_east=0)

surface_pitch = 35
azimuth_list = [90, 45, 0, -45, -90]
pv_eol = 0.88
cost = 1250
lifespan = 25
eff = 0.21
temp_coeff = -0.0048
t_ab = 0.9

max_pv = 7.5
min_pv = 2.5
pv_step_size = 0.25
pv_list = []
pv_models = {}

for i in range(int((max_pv - min_pv) / pv_step_size)):
    pv_list.append(min_pv + (i * pv_step_size))

for pv_size in pv_list:
    pv_kwp = pv_size

    array_1 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[0], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_2 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[1], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_3 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[2], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_4 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[3], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_5 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[4], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)

    # Initialize and run the SolarPVModel with all arrays
    arrays = [array_1, array_2, array_3, array_4, array_5]
    pv_model = SolarPVModel(site=site, arrays=arrays)

    pv_models[pv_size] = pv_model  # Store the model with pv_size as the key

Total_gen = 0

for model in pv_models.values():
    Total_gen += model.summary.PV_Gen_kWh_Annual

print("")
print("***************************************************")
print(f"{round(Total_gen / 1000, 3)} MWh from {len(pv_list)} models simulating {len(pv_list) * 5} PV arrays in total.")


In [ ]:
pv_models.get(2.5).models[0]["model_result"]

In [ ]:
# Access values from dictionary
pv_models.get(2.5).summary_grouped.daily.PV_Gen_kWh_Total.sum()

In [ ]:
sns.barplot(pv_models.get(2.5).summary_grouped.monthly.Cell_Temp_C_Avg - pv_models.get(2.5).summary_grouped.monthly.T2m)

In [ ]:
sns.barplot(pv_models.get(2.5).summary_grouped.weekly.PV_Gen_kWh_Total)

In [ ]:
sns.barplot(data=pv_models.get(2.5).summary_grouped.daily, x="Day_of_Year", y="PV_Gen_kWh_Total", 
            hue="PV_Gen_kWh_Total", palette="viridis")

In [ ]:
for pv_size in pv_list:
    print(f"{round(pv_models.get(pv_size).summary_grouped.daily.PV_Gen_kWh_Total.sum() / 1000, 3)} MWh/yr")

# ***VISUAL TESTING***

In [ ]:
variables = ['E_Beam_Wm2', 'E_Diffuse_Wm2', 'E_Ground_Wm2', 'E_POA_Wm2', 'ET_HRad_Wm2', 
            'Cell_Temp_C', 'PV_Gen_kWh', 'AOI', 'Zenith_Angle']

testy = pv_model.all_models

variable = "PV_Gen_kWh"

variable_1 = f"{variable}_Array_1"
variable_2 = f"{variable}_Array_2"
variable_3 = f"{variable}_Array_3"
variable_4 = f"{variable}_Array_4"
variable_7 = f"{variable}_Total"
variable_8 = f"{variable}_Avg"

if variable_1 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_1].mean())
if variable_2 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_2].mean())
if variable_3 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_3].mean())
if variable_4 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_4].mean())
if variable_7 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_7].mean(), alpha=0.4, ls="--")
if variable_8 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_8].mean(), alpha=0.4, ls="--")

In [ ]:
sns.lineplot(pv_model.combined_model["Cell_Temp_C_Avg"], alpha = 0.2, ls = "--", color = "orange")
sns.lineplot(pv_model.combined_model["T2m"])

In [ ]:
sns.barplot(pv_model.summary_grouped.daily.T2m)

In [ ]:
sns.lineplot(pv_model.combined_model.groupby("Hour_of_Day")["PV_Gen_kWh_Total"].mean())

In [ ]:
pv_model.all_models["PV_Gen_kWh_Total"].sum()

In [ ]:
pv_model.all_models.columns

In [ ]:
sns.lineplot(pv_model.all_models["PV_Thermal_Loss_kWh_Total"])

In [ ]:
temp_losses = (pv_model.all_models["PV_Thermal_Loss_kWh_Total"].sum() / pv_model.all_models["PV_Gen_kWh_Total"].sum() * 100)

print(f"{round(temp_losses, 3)}% change to power due to temperature losses")

In [ ]:
pv_model.models[0]["model_result"]

In [ ]:
pv_model.all_models

In [ ]:
pv_model.combined_model

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.daily

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.monthly

# ***MODEL ANALYSIS***